<a href="https://colab.research.google.com/github/baut-jc/ddds-c18/blob/lectures/3_2d_sqlite3_Selects_end.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1: SQL Overview & Syntax

## SQL Overview

**Structured Query Language - Databases!!**

**Why do we care?**
- Lots of data is available in SQL databases
- You may be tasked with using this data and you have to be able to get it out of the database.
- Efficient storage and retrieval of records
- Good for storing MASSIVE amounts of data.
- You may want to make your own SQL database at some point


**Lots of things you can do with SQL:**
- SELECT (get data) - This is what we'll focus on - Getting Data
- CREATE - Make tables
- INSERT - Add data to a table
- UPDATE - Update data in a table
- ALTER - Change a table
- DROP - Delete a table
- DELETE - Delete data from a table

We are going to use the following websites to practice SQL queries:
- [MySQL]( https://www.w3schools.com/mysql/trymysql.asp?filename=trysql_select_all ) <= recommended, albeit an older version
- [MS SQL Server]( https://www.w3schools.com/sql/trysqlserver.asp?filename=trysql_select_all )



Also, there are dialects of SQL between the various vendors.  For example, see [w3school's TOP](https://www.w3schools.com/sql/sql_top.asp) page.

## SQL SELECT Syntax

Sample Table


---



**people**  

| id       | name           | age | city |
| ------------- |-------------| -----|---|
| 1     | Joe | 5| Durango|
| 2      | Alice     | 55| Albuquerque |
| 3 | Veronica     |   105| Albuquerque |
|4| Matt | 43 | null |

We will use that Markdown table to create a table in SQLite3.  In order to do that, we need to install SQLite3.

In [ ]:
%%capture
%%bash
apt-get update
apt-get install -y sqlite3


In [ ]:
!sqlite3 --help
!sqlite3 --version

Usage: sqlite3 [OPTIONS] FILENAME [SQL]
FILENAME is the name of an SQLite database. A new database is created
if the file does not previously exist.
OPTIONS include:
   -A ARGS...           run ".archive ARGS" and exit
   -append              append the database to the end of the file
   -ascii               set output mode to 'ascii'
   -bail                stop after hitting an error
   -batch               force batch I/O
   -box                 set output mode to 'box'
   -column              set output mode to 'column'
   -cmd COMMAND         run "COMMAND" before reading stdin
   -csv                 set output mode to 'csv'
   -deserialize         open the database using sqlite3_deserialize()
   -echo                print commands before execution
   -init FILENAME       read/process named file
   -[no]header          turn headers on or off
   -help                show this message
   -html                set output mode to HTML
   -interactive         force interactive I/O
   -j

And to get the data into SQLite, we will first need to go through a few data wrangling steps:
1. save the Markdown table to a string
1. convert the string to HTML
1. convert the HTML to a DataFrame
1. convert the DataFrame to a CSV file
1. import the CSV file into SQLite3
1. clean up the data in SQLite3, e.g. null values, data types




In [ ]:
import pandas as pd
import markdown
import base64


In [ ]:
# Assign Markdown string to a variable

table_md = '''
| id  | name     | age | city        |
| --- | -------- | --- | ----------- |
| 1   | Joe      |   5 | Durango     |
| 2   | Alice    |  55 | Albuquerque |
| 3   | Veronica | 105 | Albuquerque |
| 4   | Matt     |  43 | null        |
'''
table_md


In [ ]:
%%markdown. #renders with (%%) as a magic command to turn into a TABLE
| id  | name     | age | city        |
| --- | -------- | --- | ----------- |
| 1   | Joe      |   5 | Durango     |
| 2   | Alice    |  55 | Albuquerque |
| 3   | Veronica | 105 | Albuquerque |
| 4   | Matt     |  43 | null        |


In [ ]:
from IPython.display import display, Markdown
display(Markdown(table_md))


In [ ]:
# Convert Markdown to HTML to a DataFrame
table_html=markdown.markdown(table_md, extensions=['markdown.extensions.tables'])
html = table_html.replace("\n",'')
html

In [ ]:
%%HTML
<table><thead><tr><th>id</th><th>name</th><th>age</th><th>city</th></tr></thead><tbody><tr><td>1</td><td>Joe</td><td>5</td><td>Durango</td></tr><tr><td>2</td><td>Alice</td><td>55</td><td>Albuquerque</td></tr><tr><td>3</td><td>Veronica</td><td>105</td><td>Albuquerque</td></tr><tr><td>4</td><td>Matt</td><td>43</td><td>null</td></tr></tbody></table>


In [ ]:
from IPython.display import display, HTML
display(HTML(html))


In [ ]:
people_df = pd.read_html(table_html)[0]
people_df


In [ ]:
# Save DataFrame as CSV
!rm -f people.*
people_df.to_csv('people.csv', index=False)
!ls -l people.*
!cat -n people.csv


In [ ]:
# Import CSV into SQLite3
%%script sqlite3 --column --header people.sql3
.mode csv
.import people.csv people


In [ ]:
# Verify files exist
!ls -la people*


In [ ]:
# Look at the schema
%%script sqlite3 --column --header people.sql3
.schema


In [ ]:
# Clean up Nulls
%%script sqlite3 --column --header people.sql3
UPDATE people SET city = NULL WHERE city = '';


In [ ]:
# Verify the data

# in sql13 a file is the database -> database(table) -> table has (columns,rows)
%%script sqlite3 --column --header people.sql3
select * from people ;
select * from people where city is null ;


## Query Foundations

**Query all rows and columns**

`SELECT columns FROM table`

`SELECT * FROM table`

*Note: as a general rule, it is better to explicitly name your columns to ensure consistent results. Naming specific columns is also better for large datasets.*

**Query specific columns**  
`SELECT name, age FROM people`

**Query based on condition**  
`SELECT name FROM people WHERE id > 1`



In [ ]:
people_df

In [ ]:
%%script sqlite3 --column --header people.sql3
-- print(people_df)
SELECT * FROM people
;


In [ ]:
%%script sqlite3 --column --header people.sql3
-- print(people_df[["name","age"]])
SELECT name, age FROM people
;


In [ ]:
%%script sqlite3 --column --header people.sql3
-- filter = (people_df["age"] > 40)
-- print(people_df[ filter ][["name"]])
SELECT name FROM people WHERE age+0 > 40  #'age+0' is to coerce age as int for sql13
;


### Your Turn
1. Select the `CustomerName` and `City` columns from `Customers`.
2. Select the `CustomerName` for people live in Sweden.
3. Select the entire row for people who live in London.

In [ ]:
# Solution 1
SELECT CustomerName, City FROM Customers;

In [ ]:
# Solution 2
SELECT CustomerName
FROM Customers
WHERE Country = 'Sweden';

In [ ]:
# Solution 3
SELECT *
FROM Customers
WHERE City = 'London'

In [ ]:
# Answer
SHOW = []
answer_b64 = b'''IyBBbnN3ZXIKIyAtLXF1ZXJ5IDEKIyBTRUxFQ1QgQ3VzdG
                9tZXJOYW1lLCBDaXR5CiMgRlJPTSBDdXN0b21lcnM7Cgoj
                IC0tcXVlcnkgMgojIFNFTEVDVCBDdXN0b21lck5hbWUKIy
                BGUk9NIEN1c3RvbWVycwojIFdIRVJFIENvdW50cnkgPSAi
                U3dlZGVuIgoKIyAtLXF1ZXJ5IDMKIyBTRUxFQ1QgKgojIE
                ZST00gQ3VzdG9tZXJzCiMgV0hFUkUgQ2l0eSA9ICdMb25k
                b24nCg=='''
if SHOW: print(base64.b64decode(answer_b64).decode('ascii'))


## And, Or, In


**AND - Combine Conditions**  
`SELECT name FROM people WHERE id > 1 AND age > 50`


**OR**  
`SELECT name FROM people WHERE name = 'Joe' OR name = 'Alice'`

**IN**  Require something to be in a list of values  
`SELECT name FROM people WHERE name IN ('Joe', 'Alice')`

**Combining AND with OR - BEWARE!! - Remember your parentheses!**

`SELECT name FROM people WHERE (id > 1 AND age > 50) AND (name = 'Joe' OR name = 'Alice')`



In [ ]:
%%script sqlite3 --column --header people.sql3
-- people_df[["name"]][ ( people_df[ "id" ] > 1 ) & ( people_df["age"] > 50)]

SELECT name
FROM people
WHERE id+0 > 1 AND age+0 > 50
;


In [ ]:
%%script sqlite3 --column --header people.sql3
-- people_df[["name"]][ ( people_df[ "name" ]  == "Joe" ) | ( people_df["name"] == "Alice" )]

SELECT name
FROM people
WHERE name = 'Joe' OR name = 'Alice'
;


In [ ]:
%%script sqlite3 --column --header people.sql3
-- people_df[["name"]][ ( people_df[ "name" ] in ( "Joe", "Alice" ) ) ]

SELECT name
FROM people
WHERE name IN ('Joe', 'Alice')
;


In [ ]:
%%script sqlite3 --column --header people.sql3

SELECT
  name
FROM
  people
WHERE
    (id+0 > 1 AND age+0 > 50)
  AND
    (name = 'Joe' OR name = 'Alice')
;


### Your Turn
From the `Customers` table ...
1. Select the entire row for people who live in Spain and have a `CustomerID` less than 20.
2. Select the entire row for people who live in either France or Germany.

In [ ]:
# Solution 1
SELECT *
FROM Customers
WHERE
    (CustomerID+0 < 20 and Country = 'Spain')

In [ ]:
# Solution 2
SELECT *
FROM Customers
WHERE
    (Country = 'France' or Country = 'Germany')

In [ ]:
# Answer
SHOW = 0
answer_b64 = b'''Ci0tIHF1ZXJ5IDEKU0VMRUNUICoKRlJPTSBDdXN0b21lcnMgCl
                dIRVJFIENvdW50cnkgPSAiU3BhaW4iIGFuZCBDdXN0b21lcklE
                IDwgMjAKOwoKLS0gcXVlcnkgMgpTRUxFQ1QgKgpGUk9NIEN1c3
                RvbWVycwpXSEVSRSBDb3VudHJ5IGluICgiRnJhbmNlIiwiR2Vy
                bWFueSIpCjs=
'''
if SHOW: print(base64.b64decode(answer_b64).decode('ascii'))


## Aliases and Wildcards

**AS - Alias a column**  
`SELECT name AS 'first_name' FROM people`

**% Wildcard - LIKE**  Get all names that start with A  
`SELECT * FROM people WHERE name LIKE  'A%'`

**_ Wildcard - LIKE**  Match any single character   
`SELECT * FROM people WHERE name LIKE  '_oe'`

[SQL Reference for LIKE Statement](https://www.w3schools.com/sql/sql_like.asp)



In [ ]:
%%script sqlite3 --column --header people.sql3
SELECT name AS 'first_name'
FROM people
;


In [ ]:
%%script sqlite3 --column --header people.sql3
SELECT *
FROM people
WHERE name LIKE 'A%'
;


In [ ]:
%%script sqlite3 --column --header people.sql3
SELECT *
FROM people
WHERE name LIKE '_oe'
;


### Your Turn
From the `Customers` table ...

1. Select the entire row for people whose `CustomerName` start with "W'
2. Select the entire row for people whose `PostalCode` ends in a 4

In [ ]:
# Solution 1 (5 results)
SELECT *
FROM Customers
WHERE CustomerName LIKE 'W%'

In [ ]:
# Solution 2 (5 results)
SELECT *
FROM Customers
WHERE PostalCode LIKE '%4'

In [ ]:
# Answer
SHOW = ""
answer_b64 = b'''Ci0tIHF1ZXJ5IDEKU0VMRUNUICoKRlJPTSBDdXN0b21lcnMKV0
                hFUkUgQ3VzdG9tZXJOYW1lIExJS0UgJ1clJwo7CgotLSBxdWVy
                eSAyClNFTEVDVCAqCkZST00gQ3VzdG9tZXJzCldIRVJFIFBvc3
                RhbENvZGUgTElLRSAnJTQnCjsK
'''
if SHOW: print(base64.b64decode(answer_b64).decode('ascii'))


## Additional Queries


**BETWEEN** Includes both the lower and upper range of the BETWEEN statement.

`SELECT name, age FROM people WHERE age BETWEEN 18 AND 70`

**LIMIT rows returned**  
`SELECT name FROM people LIMIT 5`

**OFFSET**  skips entries (i.e. page 2 of search results skips first 5 on page 1)  
`SELECT name FROM PEOPLE LIMIT 5 OFFSET 2`

**NULL** Do not use != or <> or = for NULL

--Get all records where city has some data.

`SELECT * FROM people WHERE city IS NOT NULL`


--Get all records where city has no data - not the same as "blank"

`SELECT * FROM people WHERE city IS NULL`

**ROUND**  - Rounds average age to 2 decimals  
`SELECT ROUND(AVG(age), 2) FROM people`

**NULL vs blank**

- NULL is the absence of data meaning data was never collected or provided for that data element. For example, a value may have been left off a survey. You will also get nulls when joining data if the data only exists in one table but not another.

- blank is empty data meaning data was collected but did not exist for that data element. For example, an address may not have an apartment number so that could be blank.

- Each serves a different purpose. A database administrator will decide which is appropriate for their application.

In [ ]:
filter = (people_df["age"].between(18, 70))
people_df[ filter ][["name","age"]]


In [ ]:
%%script sqlite3 --column --header people.sql3
-- filter = (people_df["age"].between(18, 70))
-- people_df[ filter ][["name","age"]]
SELECT name, age
FROM people
WHERE age+0 BETWEEN 18 AND 70
;


In [ ]:
%%script sqlite3 --column --header people.sql3
SELECT name
FROM people
LIMIT 2
;


In [ ]:
%%script sqlite3 --column --header people.sql3
SELECT name
FROM PEOPLE
LIMIT 5 OFFSET 2
;


In [ ]:
%%script sqlite3 --column --header people.sql3
SELECT *
FROM PEOPLE
;


In [ ]:
%%script sqlite3 --column --header people.sql3
SELECT *
FROM people
WHERE city IS NOT NULL
;


In [ ]:
%%script sqlite3 --column --header people.sql3
SELECT *
FROM people
WHERE city IS NULL
;


In [ ]:
%%script sqlite3 --column --header people.sql3
--
SELECT AVG(age+0)/1000 as "mean_age"
FROM people
;

SELECT Round(AVG(age+0)/1000,2) "milli_mean_age"
FROM people
;


### Your Turn
From the `Customers` table ...

1. Select the first 6 rows of people who have CustomerIDs between 20 and 35.

In [ ]:
# Solution
SELECT *
FROM Customers
WHERE CustomerID+0 BETWEEN 20 and 35
;

In [ ]:
# Answer
SHOW = []
answer_b64 = b'''ClNFTEVDVCAqCkZST00gQ3VzdG9tZXJzCldIRVJFIEN1c3RvbW
                VySUQgQkVUV0VFTiAyMCBhbmQgMzUKTElNSVQgNgo7Cg==
'''
if SHOW: print(base64.b64decode(answer_b64).decode('ascii'))


## Sorting & Grouping

**ORDER BY**  ASC (default) or DESC  
`SELECT * FROM people ORDER BY name DESC`

`SELECT * FROM people ORDER BY city, name DESC`


**DISTINCT**  Returns unique values of a column  
`SELECT DISTINCT city FROM people`

`SELECT DISTINCT city, name FROM people`


**GROUP BY**  (Can come after FROM or WHERE but must come before ORDER BY or LIMIT)  -- Total the number of people in each city

`SELECT city, COUNT(1) FROM people GROUP BY city`

**HAVING** Further filters groups-- only pull cities with more than 1 people in them  
`SELECT city, COUNT(1) FROM people GROUP BY city HAVING COUNT(1) > 1`

In [ ]:
%%script sqlite3 --column --header people.sql3
SELECT *
FROM people
ORDER BY name DESC
;


In [ ]:
%%script sqlite3 --column --header people.sql3
SELECT *
FROM people
ORDER BY city, name DESC
;


In [ ]:
%%script sqlite3 --column --header people.sql3
SELECT DISTINCT city
FROM people
;


In [ ]:
%%script sqlite3 --column --header people.sql3
SELECT COUNT(1) as count
FROM people
;


In [ ]:
%%script sqlite3 --column --header people.sql3
SELECT 1, city, COUNT(1) as count_city, count(1)
FROM people
GROUP BY city
;


In [ ]:
%%script sqlite3 --column --header people.sql3
SELECT city, COUNT(1) as count_city
FROM people
GROUP BY city
;


In [ ]:
%%script sqlite3 --column --header people.sql3
SELECT city, COUNT(1) as count_city
FROM people
GROUP BY city
HAVING count_city > 1
;


In [ ]:
%%script sqlite3 --column --header people.sql3
SELECT city, COUNT(city) as count_city
FROM people
GROUP BY city
;


In [ ]:
%%script sqlite3 --column --header people.sql3
SELECT COUNT(*)
FROM people
;


In [ ]:
%%script sqlite3 --column --header people.sql3
SELECT *
FROM people
;


### Your Turn
1. Sort the `Customers` table by `City` in alphabetical order.
2. Find the distinct countries.
3. Count the number of people in each country.

In [ ]:
# Solution
SELECT *
FROM Customers
ORDER BY City ASC
;

In [ ]:
# Solution
SELECT DISTINCT Country
FROM Customers
;

In [ ]:
# Solution (21 records)
-- Count the number of people in each country.

SELECT Country, COUNT(1) as count_country
FROM Customers
GROUP BY Country
;

In [ ]:
# Answer
SHOW = {}
answer_b64 = b'''Ci0tIHF1ZXJ5IDEKU0VMRUNUICoKRlJPTSBDdXN0b21lcnMKT1
                JERVIgQlkgQ2l0eQo7CgotLSBxdWVyeSAyClNFTEVDVCBESVNU
                SU5DVCBDb3VudHJ5CkZST00gQ3VzdG9tZXJzCjsKCi0tIHF1ZX
                J5IDMKU0VMRUNUIENvdW50cnksIENPVU5UKDEpCkZST00gQ3Vz
                dG9tZXJzCkdST1VQIEJZIENvdW50cnkKOwoK
'''
if SHOW: print(base64.b64decode(answer_b64).decode('ascii'))


## Aggregates

**COUNT(*) or COUNT(1)**  Get the number of rows *including* those with NULLs

`SELECT COUNT(*) FROM people`

**COUNT(columnName)**  Get the number of rows *excluding* those with NULLs for the specified column.

`SELECT COUNT(name) FROM people`

**MAX**  
`SELECT MAX(age) FROM people`

**MIN**  
`SELECT MIN(age) FROM people`

**SUM**  
`SELECT SUM(age) FROM people`

**AVG**  
`SELECT AVG(age) FROM people`




In [ ]:
%%script sqlite3 --column --header people.sql3
SELECT COUNT(1)
FROM people
;


71.0

In [ ]:
%%script sqlite3 --column --header people.sql3
SELECT count(city)
FROM people
;


In [ ]:
%%script sqlite3 --column --header people.sql3
SELECT MAX(age+0)
FROM people
;


In [ ]:
%%script sqlite3 --column --header people.sql3
SELECT MIN(age+0)
FROM people
;


In [ ]:
%%script sqlite3 --column --header people.sql3
SELECT SUM(age+0)
FROM people
;


In [ ]:
%%script sqlite3 --column --header people.sql3
SELECT AVG(age+0)
FROM people
;


### Your Turn
1. Find the maximum postal code in `Customers`.

In [ ]:
# Solution
SELECT MAX(PostalCode)
FROM Customers
;

In [ ]:
# Answer
SHOW = ()
answer_b64 = b'U0VMRUNUIG1heChQb3N0YWxDb2RlKSBGUk9NIEN1c3RvbWVyczs='
if SHOW: print(base64.b64decode(answer_b64).decode('ascii'))


## Aliases and CTEs

An alias makes your query output easier to read. You use them to assign meaningful names to aggregated and derived columns.

`SELECT MAX(age) AS max_age FROM people`

In [ ]:
%%script sqlite3 --column --header people.sql3
SELECT MAX(age+0) AS max_age
FROM people
;


In [ ]:
%%script sqlite3 --column --header people.sql3
SELECT p.name as "first_name", age
FROM people as "p"
WHERE age+0 > 50
;


Another type of alias is the Common Table Expression ( CTE ).  This is very similar to a context manager in Python.  A CTE allows us to create a name for a query and then use that name later in another query.

You can create a subset of your data and assign it a temporary name using the `WITH` statement.

In [ ]:
%%script sqlite3 --column --header people.sql3
WITH
  some_people as (
    SELECT name as "first_name", age+0 as age, count(1) as count
    FROM people
    GROUP by first_name, age
)

SELECT *
FROM some_people
where age > 50
;


### Your Turn
From the `Customers` table ...

1. Using a WITH statement, first select all `Customers` from Spain, then select all of those whose `CustomerID` is less than 20.

**NOTE:** MySQL < 8.0 does not support CTEs.  SQL Server requires single quotes ( not double quotes ) for strings.

In [ ]:
# Solution (Customer Name = 'Bólido Comidas preparadas')
WITH
  Spaniards as (
    SELECT *
    FROM Customers
    WHERE Country = 'Spain'
)

SELECT *
FROM Spaniards
where CustomerID+0 < 20
;

In [ ]:
# An answer
SHOW = ""
answer_b64 = b'''V0lUSAogIFNwYW5pYXJkcyBhcyAoCiAgICBTRUxFQ1QgK
                 iBGUk9NIEN1c3RvbWVycyB3aGVyZSBDb3VudHJ5ID0gIl
                 NwYWluIgopClNFTEVDVCAqIEZST00gU3BhbmlhcmRzIHd
                 oZXJlIEN1c3RvbWVySUQgPCAyMAo7'''
if SHOW: print(base64.b64decode(answer_b64).decode('ascii'))


## Encoding block

In [ ]:
def obscure(text):
  answer = text.encode("utf-8")
  b64 = base64.b64encode(answer).decode("utf-8")
  n=50
  print("\n".join([ b64[i:i+n] for i in range(0,len(b64),n) ]))

obscure('''
WITH
  Spaniards as (
    SELECT *
    FROM Customers
    WHERE Country = 'Spain'
  ),
  Madridians as (
    SELECT *
    FROM Spaniards
    WHERE City = 'Madrid'
  )
SELECT *
FROM Madridians
''')



In [ ]:
# An answer
def reveal(text, show=False):
  if SHOW: show=SHOW
  answer_b64 = text.encode("utf-8")
  if show: print(base64.b64decode(answer_b64).decode('ascii'))

SHOW = False
reveal('''CldJVEgKICBTcGFuaWFyZHMgYXMgKAogICAgU0VMRUNUICoKIC
AgIEZST00gQ3VzdG9tZXJzCiAgICBXSEVSRSBDb3VudHJ5ID0g
J1NwYWluJwogICksCiAgTWFkcmlkaWFucyBhcyAoCiAgICBTRU
xFQ1QgKgogICAgRlJPTSBTcGFuaWFyZHMKICAgIFdIRVJFIENp
dHkgPSAnTWFkcmlkJwogICkKU0VMRUNUICoKRlJPTSBNYWRyaW
RpYW5zCg==
''', True)

